In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast

In [19]:
# parameters
customer_id = '5'
formatted_attribute = 'size_diameter_rugs_runners'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')

bucket_name = """Rugs & Mats B"""

# regex_pattern = r"(?i)([^\.]\d* in\w* Belt Width)|(\d*\.\d* in\w* Belt Width)|(\d*\-\d*\/\d* in\w* Belt Width)|(\d*\.\d in\w* Belt Width)"
regex_pattern = r"(?i)(\b\s?\d+\-Inch x \d+\-Inch)|()"


value='%n/a%'
customer_name='%bedbathandbeyond%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
          'bucket_name': str(bucket_name.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [20]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)

In [21]:
df = query_from_file(file_name='query/bbby.sql', params=params)
df

DatabaseError: Execution failed on sql '

SELECT
    curated_product_fields."name" AS "attribute",
    curated_product_fields."value",
    customers."name"  AS "customers_name",
    products."external_id",
    products."name"  AS "long_desc",
    CASE WHEN curation_tasks."resolution" IS NULL THEN 'standard' ELSE resolution END
    buckets."name"  AS "buckets"
    
FROM public.curated_product_fields  AS curated_product_fields
LEFT JOIN public.curation_tasks  AS curation_tasks ON (curated_product_fields."curation_task_id") = (curation_tasks."id")
LEFT JOIN public.products  AS products ON (curated_product_fields."product_id") = (products."id")
LEFT JOIN public.customers  AS customers ON (curated_product_fields."customer_id") = (customers."id")

WHERE ((curated_product_fields."name" ) = 'size_diameter_rugs_runners')

AND (customers."name" ) LIKE '%bedbathandbeyond%'
GROUP BY
    1,
    2,
    3,
    4,
    5,
    6
ORDER BY
    1,
    2,
    3,
    4,
    5 DESC
    
    

': syntax error 

In [ ]:
# Find a list of values to iterate
df_strategy = query_from_file(file_name='./asset/easy.sql', params=params) #create query
df_strategy
list_values=df_strategy['value'].to_list()
list_values

In [ ]:
kwp=KeywordProcessor()
kwp.add_keywords_from_list(list_values)
# Extract Key Words
df['long_desc'].apply(lambda x: kwp.extract_keywords(x)) #

In [ ]:
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))

In [ ]:
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
x=df[df['matches'].astype(str)!='[]']

In [ ]:
x['matches']=x['matches'].str.get(0)
x['matches']=x['matches'].apply(lambda x:x.lower())

In [5]:
x

NameError: name 'x' is not defined

In [6]:
x[['length', 'width']]=x['matches'].str.split(" x ",expand=True,)


NameError: name 'x' is not defined

In [7]:
x['length']=x['length'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\s|\w*\.\w*\.|\-','',str(measure)))
x['width']=x['width'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\s|\w*\.\w*\.|\-','',str(measure)))

NameError: name 'x' is not defined

In [8]:
x['length'].dtypes

NameError: name 'x' is not defined

In [9]:
x['length']=x['length'].astype(int)
x['width']=x['width'].astype(int)
x['Max'] = x[['length', 'width']].values.max(1)
x['Min'] = x[['length', 'width']].values.min(1)
x

NameError: name 'x' is not defined

In [10]:
# y['length'],['width']=x['matches'].str.split(" x ",expand=True,)
# y

In [11]:
# error

In [12]:
# y=x['matches'].str.split(" x ",expand=True,)
# y

In [18]:
# x['matches']=x['matches'].astype(str)
# x.dtypes

In [19]:
# y=pd.DataFrame(x["matches"].str.split(' x ', expand=True).values,
#              columns=['length', 'width'])
# y
# y=x['matches'].str.split(" x ",expand=True,)
# y

In [20]:
# x=df['matches'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\s|\w*\.\w*\.|\-','',str(measure)))
# x

In [21]:
# regex_pattern = r'(\d{1,}\.?\d*\\?\")|(\d{1,}\/?\d*\\?\")|(\d{1,}\.?\d*\\?\'\w*\W)|(\d{1,}\Wf\w{0,2}t)|(\d{1,}\Winche?s?)'
# df['measurements'] = df['long_desc'].apply(lambda x: extract_matches(regex_pattern, x))    

In [22]:
# def write_out_to_num(txt):
#     measure = ''
#     try:
#         txt = txt.lower().split("x")[1].strip()
#     except:
#         txt = txt 
#     if len(re.findall("f\w{0,2}t",txt)) > 0 and txt.find("inch") >-1:                                                                                                                 
#         match_0 = txt.strip().split(" ")[0]  #inches
#         match_1 = txt.strip().split(" ")[1]  #ft
#         foot = float(re.sub(r'(?i)\W?[a-z]','',match_0))*12                    
#         inch = float(re.sub(r'(?i)\Win\w*','',match_1))                        
#         measure = int(foot) + int(inch) 
        
#     if len(re.findall("f\w{0,2}t",txt)) > 0 and txt.find("inch") == -1:
#         foot = float(re.sub(r'\W?[a-z]','',txt))*12
#         inch = 0.0
#         measure = int(foot) + int(inch) 
        
#     if len(re.findall("f\w{0,2}t",txt)) == 0and txt.find("inch") > -1:
#         foot = 0.0
#         inch = float(re.sub(r'(?i)\Winch\w*','',txt))
#         measure = int(foot) + int(inch) 
#     return f'{measure} in'

In [23]:
# f = lambda x: x["matches"].str.split(" x ")
# x['matches'] = x.apply(f, axis=1)

In [24]:
# y = pd.DataFrame(x.str.split(' x ',1).tolist(),
#                                  columns = ['length','width'])

In [25]:
# def parse_ht(ht):
#     # format: 7' 0.0"
#     length, width= ht.split(" x ")
# #     ft_ = float(ht_[0])
# #     in_ = float(ht_[1].replace("\"",""))

#     return length, width

In [26]:
# x=pd.DataFrame()
# x=df['matches'].apply(lambda s: parse_ht(s))

In [27]:
# x=df['matches'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\s|\w*\.\w*\.|\-','',str(measure)))
# x

In [28]:
# error

In [29]:
# regex_pattern = r'(\d{1,}\.?\d*\\?\")|(\d{1,}\/?\d*\\?\")|(\d{1,}\.?\d*\\?\'\w*\W)|(\d{1,}\Wf\w{0,2}t)|(\d{1,}\WInche?s?)'
# df['measurements'] = df['long_desc'].apply(lambda x: extract_matches(regex_pattern, x))  
# df

In [30]:
# def write_out_to_num(txt):
#     measure = ''
#     try:
#         txt = txt.lower().split("x")[1].strip()
#     except:
#         txt = txt 
#     if len(re.findall("f\w{0,2}t",txt)) > 0 and txt.find("inch") >-1:                                                                                                                 # find inch but how does this find the feet?
#         match_0 = txt.strip().split(" ")[0]  #inches
#         match_1 = txt.strip().split(" ")[1]  #ft
#         foot = float(re.sub(r'(?i)\W?[a-z]','',match_0))*12                    #substitute match_0 with the number of ft*12 inches
#         inch = float(re.sub(r'(?i)\Win\w*','',match_1))                        #get number of inches
#         measure = int(foot) + int(inch) 
        
#     if len(re.findall("f\w{0,2}t",txt)) > 0 and txt.find("inch") == -1:
#         foot = float(re.sub(r'\W?[a-z]','',txt))*12
#         inch = 0.0
#         measure = int(foot) + int(inch) 
        
#     if len(re.findall("f\w{0,2}t",txt)) == 0and txt.find("inch") > -1:
#         foot = 0.0
#         inch = float(re.sub(r'(?i)\Winch\w*','',txt))
#         measure = int(foot) + int(inch) 
#     return f'{measure} in'

In [31]:
# x=x.astype(str)
# x.dtype

In [32]:
# x.apply(lambda split: write_out_to_num(split))

In [33]:
# x['cleaned']=df['matches'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\s|\w*\.\w*\.|\-','',str(measure)))
# max(pd.to_numeric(x['matches'][0:4], errors='coerce'))

In [34]:
# matches['matches'] = matches['matches'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\s|\w*\.\w*\.','',str(measure)))

In [35]:
# error

In [36]:
# df['max']=pd.to_numeric(df['matches'], errors='coerce')
# df

In [37]:
# df['matches']=df[df['matches']!='[]']

In [38]:
# # df['Max']=pd.DataFrame(df['matches'].tolist(),index=df.index).max(1)
# y=df[1:4]
# # z=y['matches']
# maximum=max(y['matches'])
# maximum=pd.to_numeric(maximum)
# print(max(maximum))

In [39]:
# x=df
# x=x[x['matches']!='[]']
# x['Max']=pd.DataFrame(df['matches'].tolist(),index=df.index).max(1) 
# x

In [40]:
# x['matches'].idxmax()

In [41]:
# df['matches'] = pd.to_numeric(df['matches'])

In [42]:
# df.loc[df['matches'].idxmax()]

In [43]:
# print(df[df['matches']==df['matches'].max()])

In [15]:
matches = df[df['matches'].astype(str) != '[]'] 
x=matches[matches['value']=='n/a']
x

NameError: name 'df' is not defined

In [16]:
def remove_txt(lst):
    assert type(lst) == list, "make sure it's a list"
    tmp = []
    for measure in lst:
        val = re.sub(r'[a-z]|[A-z]|\s|\w*\.\w*\.','',measure)
        if val != '':
            tmp.append(val.replace(",",''))
    if str(tmp) == '[]':
        return ''
    else:
        return '{:,}'.format(int(max(tmp)))

In [17]:
matches['matches'] = matches['matches'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\s|\w*\.\w*\.','',str(measure)))

NameError: name 'matches' is not defined

In [18]:
matches['matches'].unique()

NameError: name 'matches' is not defined

In [ ]:
def remove_fractions(numbers):
    if numbers.find('-') > -1:
        numbers = literal_eval(numbers)
        inte, fract = numbers.split('-')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
        return str(num)
    else:
        return numbers

In [ ]:
def remove_fraction(numbers):
    if numbers.find('/') > -1:
        numbers = literal_eval(numbers)
#         inte, fract = numbers.split('-')
        nom, den = numbers.split('/')
        tmp = float(nom)/float(den)
#         num = int(inte) + tmp
        return str(tmp)
    else:
        return numbers

In [ ]:
matches['matches'] = matches['matches'].apply(lambda x: remove_fractions(x))

In [ ]:
matches['matches'].unique()

In [ ]:
matches['matches'] = matches['matches'].apply(lambda x: remove_fraction(x))

In [ ]:
matches['matches'].unique()

In [ ]:
matches['matches'].value_counts()

In [ ]:
matches

In [ ]:
matches['matches'] = matches['matches'].apply(lambda x: float(literal_eval(x)))
matches

In [ ]:
matches[curation_col]=matches['matches']
matches

In [ ]:
error

In [ ]:
gc = gcloud.Gsheets()

In [ ]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',matches)

# send to the folder for upload

In [ ]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload'
    non_matches = df[df['matches'].astype(str) == '[]']
    non_matches[curation_col] = r'n/a'
    non_matches.to_csv(f'{drive_path}/Motion - {attribute} - na upload_Pulley.csv',index=False)
    matches.to_csv(f'{drive_path}/Motion - {attribute} - match upload_Shawn.csv',index=False)

In [ ]:
looks_good('Motion Industries', attribute, df, matches)